In [13]:
import numpy as np
from networkx.classes.function import common_neighbors
import csv
import networkx as nx
import sqlite3
import pandas as pd
import pickle
import json

In [2]:
clean_rows = []

In [3]:
for row in csv.reader(open('kudos.csv', 'r')):
    clean_rows.append([eval(x).decode() for x in row])

In [4]:
clean_rows = [x for x in clean_rows if len(x) == 3]

In [9]:
g = nx.Graph()
users=set()
fics=set()
fic_dict = {}
for row in clean_rows:
    fic = row[1]
    if fic not in fic_dict:
        fic_dict[row[1]] = row[0]
    user = row[2]
    g.add_edge(fic, user)
    users.add(user)
    fics.add(fic)
fics=np.array(list(fics))


In [54]:
cov=np.zeros(len(fics))
target='27268831'
for i in range(0,len(fics)):
    if not target== fics[i]:
        cov[i]=sum(2 for i in common_neighbors(g, fics[i], target))/(len(g[fics[i]])+len(g[target]))


In [58]:
num_recs=20

In [63]:
top_n_fics=fics[np.flip(np.argsort(cov))[0:num_recs]]
top_n_scores=cov[np.flip(np.argsort(cov))[0:num_recs]]
top_n_urls=np.core.defchararray.add('https://archiveofourown.org/works/',top_n_fics)


In [64]:
top_n_titles=[]
for id in top_n_fics:
    top_n_titles.append(fic_dict[id])

In [65]:
df = pd.DataFrame(list(zip(top_n_titles, top_n_urls, top_n_scores )), columns =['title', 'url', 'score']) 
print("Top Recomendations for "+fic_dict[target])
df

Top Recomendations for break me like a promise


,title,url,score
0,If You Like Piña Coladas (And Getting Caught I...,https://archiveofourown.org/works/28421556,0.396226
1,keep me in your glow,https://archiveofourown.org/works/28992594,0.393134
2,just a summer thing,https://archiveofourown.org/works/27360778,0.369589
3,I Hear The Wind Call Your Name,https://archiveofourown.org/works/29870277,0.357247
4,smile like a switchblade,https://archiveofourown.org/works/26522239,0.351539
5,the cat says,https://archiveofourown.org/works/28195053,0.350061
6,a strange disease like me,https://archiveofourown.org/works/28479213,0.349682
7,The Night Shift,https://archiveofourown.org/works/25909702,0.346911
8,glory and gore,https://archiveofourown.org/works/25002529,0.345905
9,She's A Keeper,https://archiveofourown.org/works/25554244,0.343027


In [16]:
np.save("../data/fics.npy", fics)
nx.write_gpickle(g, '../data/g.gpickle')
with open('../data/fic_dict.json', 'w') as fout:
    json.dump(fic_dict, fout)